In [ ]:
#모듈불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
#장비
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

In [ ]:
#HyperParameter
batch_size = 50
epoch_num = 15
learning_rate = 0.0001

In [ ]:
#데이터 불러오기
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

print('number of training data: ', len(train_data))
print('number of test data: ', len(test_data))

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

number of training data:  60000
number of test data:  10000


In [ ]:
#미니 배치 구성하기
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)

In [ ]:
#CNN모델 구성하기_기초
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    
    self.conv1 = nn.Conv2d(1, 32, 3, 1) #(in_channels, out_channnels, kernel_size, stride)
    self.conv2 = nn.Conv2d(32, 64, 3, 1)
    self.fc1 = nn.Linear(9216, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)

    x = F.max_pool2d(x,2)

    x = torch.flatten(x,1)

    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)

    output = F.log_softmax(x, dim=1)
    return output

In [ ]:
#Optimizer 및 손실 함수 정의
model= CNN().to(device)
optimizer= optim.Adam(model.parameters(), lr= learning_rate)
criterion= nn.CrossEntropyLoss()#분류

In [ ]:
print(model)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [ ]:
#모델학습
model.train()
i=0
for epoch in range(epoch_num):
  for data, target in train_loader:
    data = data.to(device)
    target = target.to(device)
    
    optimizer.zero_grad()#가중치 초기화
    
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    if i %1000==0:
      print('Train step: {}\tloss: {:,.3f}'.format(i, loss.item()))
    i+=1

Train step: 0	loss: 2.301
Train step: 1000	loss: 0.323
Train step: 2000	loss: 0.074
Train step: 3000	loss: 0.090
Train step: 4000	loss: 0.178
Train step: 5000	loss: 0.094
Train step: 6000	loss: 0.005
Train step: 7000	loss: 0.011
Train step: 8000	loss: 0.029
Train step: 9000	loss: 0.133
Train step: 10000	loss: 0.003
Train step: 11000	loss: 0.044
Train step: 12000	loss: 0.048
Train step: 13000	loss: 0.004
Train step: 14000	loss: 0.002
Train step: 15000	loss: 0.023
Train step: 16000	loss: 0.012
Train step: 17000	loss: 0.006
